In [ ]:
if __name__ == '__main__':
    query_set = 1
    query_utt = 9
    # K-nearest neighbor 매개 변수 
    K = 3
    # MFCC 차원 수 
    num_dims = 13
    # 총 세트 수 
    num_sets = 5
    # 바화 종류 수 
    num_utt = 10
    
    # 특징값 데이터를 특징값 파일에서 읽어온다 
    query_file = './mfcc/REPEAT500_set%d_%03d.bin' % (query_set, query_utt)
    query = np.fromfile(query_file, dtype = np.float32)
    query = query.reshape(-1, num_dims)
    
    cost = []
    for set_id in range(1, num_set + 1):
        for utt_id in range(1, num_utt + 1):
            # query와 동일 세트는 비교하지 않는다 
            if set_id == query_set:
                continue
                
            # 비교대상의 특징값을 읽어온다
            target_file = './mfcc/REPEAT500_set%d_%03d.bin' % (set_id, utt_id)
            target = np.fromfile(target_file, dtype = np.float32)
            target = target.reshape(-1, num_dims)
            
            # DP Matching을 실시
            tmp_cost, tmp_path = dp_matching(query, target)
            cost.append({'utt': utt_id, 
                        'set': set_id, 
                        'cost': tmp_cost})
            
    # 비용 순으로 정렬한다
    cost = sorted(cost, key = lambda x: x['cost'])
    # 비용 순으로 표시한다 
    for n in range(len(cost)):
        print('%d: utt: %d, set: %d, cost: %.3f' % 
             (n+1, 
             cost[n]['utt'], 
             cost[n]['set'], 
             cost[n]['cost']))
        
    # K-nearest neighbor를 수행한다
    voting = np.zeros(num_utt, np.int16)
    for n in range(K):
        # 상위 k개 발화 ID로 투표를 진행한다
        voting[cost[n]['utt']-1] += 1
    
    # 투표로 가장 많은 값을 얻은 발화 ID를 출력한다
    max_voted = np.argmax(voting) + 1
    print('Estimated utterange id = %d' % max_voted)
    

In [ ]:
if __name__ == '__main__':
    
    num_set = 5
    num_utt = 10
    
    query_set = 1
    query_utt = 9
    
    num_dims = 13
    K = 3
    
    query_file = './mfcc/REPEAT500_set%d_%03d.bin' % (query_set, query_utt)
    query = np.fromfile(query_file, dtype = np.float32)
    query = query.reshape(-1, num_dims)
    
    cost = []
    for set_id in range(1, num_set + 1):
        for utt_id in range(1, num_utt + 1):
            if set_id == query_set:
                continue 
            target_file = './mfcc/REPEAT500_set%d_%03d.bin' % (set_id, utt_id)
            target = np.fromfile(target_file, dtype = np.float32)
            target = target.reshape(-1, num_dims)
            
            tmp_cost, tmp_path = dp_matching(query, target)
            cost.append({'utt': utt_id, 
                        'set': set_id, 
                        'cost': tmp_cost})
            
    cost = sorted(cost, key = lambda x: x['cost'])
    for n in range(len(cost)):
        print('%d: utt: %d, set: %d, cost: %.3f' % 
             (n+1, 
             cost[n]['utt'], 
             cost[n]['set'], 
             cost[n]['cost']))
    
    voting = np.zeros(num_utt, np.int16)
    for n in range(K):
        voting[cost[n]['utt'] -1] += 1
    max_voted = np.argmax(voting) + 1
    print('Estimated utterance id = %d' % max_voted)